# Neural Network

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv("prostate_cancer_prediction.csv")
data

,Patient_ID,Age,Family_History,Race_African_Ancestry,PSA_Level,DRE_Result,Biopsy_Result,Difficulty_Urinating,Weak_Urine_Flow,Blood_in_Urine,...,Alcohol_Consumption,Hypertension,Diabetes,Cholesterol_Level,Screening_Age,Follow_Up_Required,Prostate_Volume,Genetic_Risk_Factors,Previous_Cancer_History,Early_Detection
0,1,78,No,Yes,5.07,Normal,Benign,No,No,No,...,Moderate,No,No,Normal,45,No,46.0,No,No,Yes
1,2,68,No,Yes,10.24,Normal,Benign,Yes,No,No,...,Low,No,No,High,65,No,78.2,No,No,Yes
2,3,54,No,No,13.79,Normal,Benign,No,No,No,...,Low,No,No,Normal,61,No,21.1,No,No,Yes
3,4,82,No,No,8.03,Abnormal,Benign,No,No,No,...,Low,No,No,Normal,47,Yes,79.9,No,Yes,Yes
4,5,47,Yes,No,1.89,Normal,Malignant,Yes,Yes,No,...,Moderate,Yes,No,Normal,72,No,32.0,No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27940,27941,49,Yes,No,9.35,Normal,Benign,No,No,No,...,High,Yes,Yes,Normal,42,No,47.9,No,No,Yes
27941,27942,84,Yes,No,1.24,Normal,Benign,No,No,No,...,Moderate,Yes,No,Normal,47,Yes,55.3,No,No,Yes
27942,27943,69,No,No,5.01,Abnormal,Malignant,No,No,No,...,Low,Yes,No,Normal,44,No,47.0,No,Yes,Yes
27943,27944,50,No,No,5.71,Normal,Benign,Yes,Yes,No,...,Moderate,Yes,Yes,Normal,67,No,24.2,No,No,Yes


In [4]:

# Drop non-predictive features
data.drop('Patient_ID', axis=1, inplace=True)

# Encode the target variable (Biopsy_Result)
data['Biopsy_Result'] = data['Biopsy_Result'].map({'Benign': 0, 'Malignant': 1})

# One-hot encode categorical variables
categorical_cols = data.select_dtypes(include='object').columns
encoder = OneHotEncoder(drop='first', dtype=int)
encoded_data = pd.DataFrame(encoder.fit_transform(data[categorical_cols]).toarray(),
                            columns=encoder.get_feature_names_out(categorical_cols))

# Drop original categorical columns and combine encoded data
data.drop(categorical_cols, axis=1, inplace=True)
data = pd.concat([data, encoded_data], axis=1)

# Scale numerical features
numerical_cols = ['Age', 'PSA_Level', 'BMI', 'Screening_Age', 'Prostate_Volume']
scaler = StandardScaler()
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

# Split data into training and testing sets
X = data.drop('Biopsy_Result', axis=1)
y = data['Biopsy_Result']
X


,Age,PSA_Level,BMI,Screening_Age,Prostate_Volume,Family_History_Yes,Race_African_Ancestry_Yes,DRE_Result_Normal,Difficulty_Urinating_Yes,Weak_Urine_Flow_Yes,...,Smoking_History_Yes,Alcohol_Consumption_Low,Alcohol_Consumption_Moderate,Hypertension_Yes,Diabetes_Yes,Cholesterol_Level_Normal,Follow_Up_Required_Yes,Genetic_Risk_Factors_Yes,Previous_Cancer_History_Yes,Early_Detection_Yes
0,0.939988,-0.642309,-0.861585,-1.176363,-0.093872,0,1,1,0,0,...,1,0,1,0,0,1,0,0,0,1
1,0.245761,0.596033,-1.250276,0.800335,1.627690,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,1
2,-0.726158,1.446345,-1.229818,0.404995,-1.425141,0,0,1,0,0,...,1,1,0,0,0,1,0,0,0,1
3,1.217679,0.066684,0.386317,-0.978694,1.718580,0,0,0,0,0,...,0,1,0,0,0,1,1,0,1,1
4,-1.212117,-1.403997,0.734093,1.492179,-0.842377,1,0,1,1,1,...,0,0,1,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27940,-1.073272,0.382856,0.488604,-1.472868,0.007711,1,0,1,0,0,...,1,0,0,1,1,1,0,0,0,1
27941,1.356525,-1.559688,-0.984330,-0.978694,0.403350,1,0,1,0,0,...,0,0,1,1,0,1,1,0,0,1
27942,0.315184,-0.656680,0.140828,-1.275198,-0.040407,0,0,0,0,0,...,0,1,0,1,0,1,0,0,1,1
27943,-1.003849,-0.489013,-1.086617,0.998004,-1.259401,0,0,1,1,1,...,0,0,1,1,1,1,0,0,0,1


In [5]:
# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Convert to tensors
X_train_tensor, y_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device), torch.tensor(y_train.values, dtype=torch.float32).to(device)
X_val_tensor, y_val_tensor = torch.tensor(X_val.values, dtype=torch.float32).to(device), torch.tensor(y_val.values, dtype=torch.float32).to(device)
X_test_tensor, y_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device), torch.tensor(y_test.values, dtype=torch.float32).to(device)

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)


Using device: cpu


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [7]:
# Initialize model, loss function, and optimizer
model = NeuralNetwork(X_train.shape[1]).to(device)
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, initial_lr=0.004, patience=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()

            outputs = model(inputs).squeeze() 

            loss = criterion(outputs, targets) 

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation step
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs).squeeze()
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        scheduler.step(avg_val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

    model.load_state_dict(torch.load("best_model.pth"))

    # Plot training and validation loss
    plt.figure(figsize=(10,5))
    plt.plot(range(len(train_losses)), train_losses, label="Train Loss")
    plt.plot(range(len(val_losses)), val_losses, label="Validation Loss", linestyle="dashed")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Training and Validation Loss Over Epochs")
    plt.show()

    return model


# Evaluation loop
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            outputs = model(X_batch).squeeze()
            predictions = (outputs >= 0.5).float()
            correct += (predictions == y_batch).sum().item()
            total += y_batch.size(0)
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy * 100:.2f}%")



In [9]:
# Train and evaluate the model
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=100)
evaluate(model, test_loader)

Epoch 1/100, Train Loss: 0.6041, Val Loss: 0.6134
Epoch 2/100, Train Loss: 0.6040, Val Loss: 0.6134
Epoch 3/100, Train Loss: 0.6046, Val Loss: 0.6134
Epoch 4/100, Train Loss: 0.6045, Val Loss: 0.6134
Epoch 5/100, Train Loss: 0.6042, Val Loss: 0.6134
Epoch 6/100, Train Loss: 0.6039, Val Loss: 0.6134
Epoch 7/100, Train Loss: 0.6044, Val Loss: 0.6134
Epoch 8/100, Train Loss: 0.6041, Val Loss: 0.6134
Epoch 9/100, Train Loss: 0.6044, Val Loss: 0.6134
Epoch 10/100, Train Loss: 0.6043, Val Loss: 0.6134
Epoch 11/100, Train Loss: 0.6041, Val Loss: 0.6134
Epoch 12/100, Train Loss: 0.6047, Val Loss: 0.6134
Epoch 13/100, Train Loss: 0.6046, Val Loss: 0.6134
Epoch 14/100, Train Loss: 0.6040, Val Loss: 0.6134
Epoch 15/100, Train Loss: 0.6046, Val Loss: 0.6134
Epoch 16/100, Train Loss: 0.6041, Val Loss: 0.6134
Epoch 17/100, Train Loss: 0.6042, Val Loss: 0.6134
Epoch 18/100, Train Loss: 0.6046, Val Loss: 0.6134
Epoch 19/100, Train Loss: 0.6046, Val Loss: 0.6134
Epoch 20/100, Train Loss: 0.6044, Val Lo

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

## Model Optimization

In this section, we will attempt to use Stochastic Gradient Descent (SGD) with momentum to try and optimize the neural network. This method of optimization updates the model’s weights using random subsets of data, making it more efficient than batch gradient descent. Adding momentum (set to 0.9) helps accelerate convergence by smoothing updates and reducing oscillations. We set the learning rate to 0.01 to control the step size in each update.

Now, let's define the model, apply the optimizer, and test it again.

In [19]:
# instantiate the model (using the same parameters as before)
input_size = 34
hidden_size1 = 64
hidden_size2 = 32
output_size = 1

model = NeuralNetwork(input_size, hidden_size1, hidden_size2, output_size)

# define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# define loss function
criterion = nn.MSELoss()

# train the new model
def train_model(model, train_loader, optimizer, criterion, epochs=10):
    model.train() 
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')

# test the model
def test_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad(): 
        for inputs, labels in test_loader:
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels) 
            total_loss += loss.item()

            predictions = torch.round(outputs)  # round to nearest integer for classification
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(test_loader)
    accuracy = correct / total * 100
    print(f'Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%')

    return accuracy, avg_loss


# train the model
train_model(model, train_loader, optimizer, criterion, epochs=10)

# test the model
test_accuracy, test_loss = test_model(model, test_loader, criterion)
print(f'Test Accuracy after optimization: {test_accuracy:.2f}%, Test Loss: {test_loss:.4f}')


Epoch [1/10], Loss: 0.2194
Epoch [2/10], Loss: 0.2110
Epoch [3/10], Loss: 0.2112
Epoch [4/10], Loss: 0.2104
Epoch [5/10], Loss: 0.2104
Epoch [6/10], Loss: 0.2100
Epoch [7/10], Loss: 0.2101
Epoch [8/10], Loss: 0.2101
Epoch [9/10], Loss: 0.2100
Epoch [10/10], Loss: 0.2100
Test Loss: 0.2117, Test Accuracy: 69.75%
Test Accuracy after optimization: 69.75%, Test Loss: 0.2117
